In [1]:
#import 
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np


#import des files csv : 
#fr_animaux.csv : multiples indicateurs de production des produits animaux en 2013
df_animaux  = pd.read_csv ('FAO_2013_fr/fr_animaux.csv')         

#fr_céréales.csv: quantité de céréales produites au niveau mondial en 2013   
df_cereales = pd.read_csv ('FAO_2013_fr/fr_cereales.csv')  

#fr_population.csv: population mondiale par pays en 2013
df_pop = pd.read_csv ('FAO_2013_fr/fr_population.csv')       

#fr_sousalimentation.csv: nombre de personnes sous alimentées dans le monde de 2013 à 2017
df_sousalim = pd.read_csv ('FAO_2013_fr/fr_sousalimentation.csv')

#fr_vegetaux.csv: multiples indicateurs de production des produits végétaux en 2013
df_vegetaux = pd.read_csv ('FAO_2013_fr/fr_vegetaux.csv')   

In [2]:
#ajout de colonne
df_animaux['origine'] = 'animaux'
df_cereales['origine'] = 'cereales'
df_vegetaux['origine'] = 'vegetaux'

#creation d un df avec 'animaux' + 'cereales' + 'vegetaux'
data = df_animaux[['Zone' , 'Code zone' , 'Produit' ,'Code Produit', 'Élément' , 'Valeur' , 'Unité', 'Année' ,'origine' ]].append ( df_vegetaux[['Zone' ,  'Code zone' , 'Produit','Code Produit' , 'Élément' , 'Valeur' , 'Unité' , 'Année' ,'origine' ]] )
#data = data.append ( df_vegetaux[['Zone' ,  'Code zone' ,  'Produit' ,'Code Produit', 'Élément' , 'Valeur' , 'Unité' , 'Année' , 'origine']] )
data ['Info'] = data ['Élément'] + ' (' + data ['Unité'] + ')'  #fusion de colonne 

#pivot table  
df = data.pivot_table ( values=  ['Valeur'] ,
                        index=['Zone','Code zone' , 'Année' , 'origine','Code Produit','Produit'],
                        columns=['Info'],
                        aggfunc=sum,)

liste_colonne = df.iloc[0].reset_index()['Info']   #recuperer les noms des colonnes
df.columns = [f'{i}' for i in liste_colonne]  #changer le nom des colonnes
df = df.reset_index()  

#integration de la population 
df_pop ['Population']  = df_pop ['Valeur']  * 1000
df = df.merge (    df_pop [['Zone', 'Population' ]] )

df_sousalim["Valeur"].replace ("<0.1", '0.1',inplace =True)   #valeur incoherente '<0.1'
df_sousalim['Valeur'] = pd.to_numeric(df_sousalim['Valeur'])# to float

df_ss_al = df_sousalim[df_sousalim['Année'] == '2012-2014']   #l annee 2013 

df_ss_al['Pop sous alimente'] = df_ss_al['Valeur'] * 1000000   #Creation de la colonne 
df = df.merge (    df_ss_al [['Zone', 'Pop sous alimente' ]] )  #merge

#rename les columns
df.rename ( columns =  {'Disponibilité alimentaire (Kcal/personne/jour) (Kcal/personne/jour)': 'Disponibilité alimentaire (Kcal/personne/jour)',
                       'Disponibilité alimentaire en quantité (kg/personne/an) (kg)' : 'Disponibilité alimentaire (kg/personne/an)',
                       'Disponibilité de matière grasse en quantité (g/personne/jour) (g/personne/jour)' : 'Disponibilité de matière grasse (g/personne/jour)',
                       'Disponibilité de protéines en quantité (g/personne/jour) (g/personne/jour)' : 'Disponibilité de protéines (g/personne/jour)',
                       'Exportations - Quantité (Milliers de tonnes)' : 'Exportations (Milliers de tonnes)',
                       'Importations - Quantité (Milliers de tonnes)' : 'Importations (Milliers de tonnes)',
                       'Autres utilisations (non alimentaire) (Milliers de tonnes)' : 'Utilisations non alimentaire (Milliers de tonnes)'} , inplace = True )

df.head()

<ipython-input-2-e73883bd7cfb>:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ss_al['Pop sous alimente'] = df_ss_al['Valeur'] * 1000000   #Creation de la colonne


,Zone,Code zone,Année,origine,Code Produit,Produit,Aliments pour animaux (Milliers de tonnes),Utilisations non alimentaire (Milliers de tonnes),Disponibilité alimentaire (Kcal/personne/jour),Disponibilité alimentaire (kg/personne/an),...,Exportations (Milliers de tonnes),Importations (Milliers de tonnes),Nourriture (Milliers de tonnes),Pertes (Milliers de tonnes),Production (Milliers de tonnes),Semences (Milliers de tonnes),Traitement (Milliers de tonnes),Variation de stock (Milliers de tonnes),Population,Pop sous alimente
0,Afghanistan,2,2013,animaux,2731,Viande de Bovins,NaN,NaN,27.0,4.59,...,NaN,6.0,140.0,NaN,134.0,NaN,NaN,NaN,30552000,7900000.0
1,Afghanistan,2,2013,animaux,2732,Viande d'Ovins/Caprins,NaN,NaN,31.0,4.92,...,NaN,NaN,150.0,NaN,150.0,NaN,NaN,NaN,30552000,7900000.0
2,Afghanistan,2,2013,animaux,2734,Viande de Volailles,NaN,NaN,9.0,2.45,...,NaN,48.0,75.0,NaN,26.0,NaN,NaN,NaN,30552000,7900000.0
3,Afghanistan,2,2013,animaux,2735,"Viande, Autre",NaN,NaN,1.0,0.38,...,NaN,0.0,12.0,NaN,12.0,NaN,NaN,NaN,30552000,7900000.0
4,Afghanistan,2,2013,animaux,2736,Abats Comestible,NaN,NaN,5.0,1.72,...,NaN,NaN,53.0,NaN,53.0,NaN,NaN,NaN,30552000,7900000.0


# Question 1 :  

Donnez le résultat de votre calcul pour l'année 2013.

In [3]:
#calcul de la population mondiale
pop_monde = df ['Population'].unique() .sum()
print ( 'Il y a une population mondiale {} d habitant sur la planete '.format (pop_monde) )

print ('\nCe chiffre  est bien trop eleve, nous allons rechercher les pays avec une pop > 1 milliard :')

print (df [df ['Population'] > 1000000000] ['Zone'].unique() )

print ('\nLa chine est compté 2 fois')

#suppression de la chine
df = df[df['Zone'] != 'Chine']

#calcul de la population mondiale
pop_monde = df ['Population'].unique() .sum()
print ( '\nIl y a une population mondiale {} d habitant sur la planete '.format (pop_monde) )

Il y a une population mondiale 8413993000 d habitant sur la planete 

Ce chiffre  est bien trop eleve, nous allons rechercher les pays avec une pop > 1 milliard :
['Chine' 'Chine, continentale' 'Inde']

La chine est compté 2 fois

Il y a une population mondiale 6997326000 d habitant sur la planete 


# Question 2 :

Identifiez ces redondances, en donnant votre réponse sous forme de formule mathématique (pas besoin de coder ici :soleil: ). 

C'est une équation à 3 termes de type \(a_1 + a2 + [...] = b_1 + b_2 + [...] = c_1 + c_2 + [...]\) ) faisant intervenir chacune des 11 quantités données ci dessus. 

Illustrez cette équation avec l'exemple du blé en France. 
Pour avoir un indice, cliquez sur "Définitions et Standards" sur la page de téléchargement des données.

In [4]:
#creation du df temporaire et des colonnes Gestion, Dispo, Utilisation
temp = df
temp["Gestion"] = df['Production (Milliers de tonnes)'] + df['Importations (Milliers de tonnes)'] - df['Exportations (Milliers de tonnes)'] + df['Variation de stock (Milliers de tonnes)']
temp["Dispo"] = df['Disponibilité intérieure (Milliers de tonnes)']
temp["Utilisation"] = df['Aliments pour animaux (Milliers de tonnes)'] + df['Utilisations non alimentaire (Milliers de tonnes)'] + df['Nourriture (Milliers de tonnes)'] + df['Traitement (Milliers de tonnes)']+ df['Semences (Milliers de tonnes)'] + df['Pertes (Milliers de tonnes)']

temp.fillna (0, inplace = True)

print ('\ngestion - dispo : \n')
print((temp['Gestion'] - temp['Dispo']).sort_values())
print ('\ndispo - utilisation : \n')
print((temp['Dispo'] - temp['Utilisation']).sort_values())


gestion - dispo : 

2113    -739267.0
6319    -341200.0
13891   -100096.0
9190     -61181.0
3449     -54643.0
           ...   
6313         62.0
5027         90.0
13860        92.0
6745        105.0
10984       151.0
Length: 15605, dtype: float64

dispo - utilisation : 

15489     -3430.0
6433       -178.0
7145       -159.0
10984      -151.0
762        -113.0
           ...   
6305     133443.0
15461    292776.0
6319     341200.0
3439     500070.0
2113     739267.0
Length: 15605, dtype: float64


In [5]:
# utilisation pour le blé de france
temp_ble = temp[ (temp['Produit'] == 'Blé') & (temp['Zone'] == 'France' )]

print ('Pour le Blé : ')
print ('\ngestion - dispo =', (temp_ble['Gestion'] - temp_ble['Dispo']).sort_values(ascending = False).sum())
print ('\ndispo - utilisation =',(temp_ble['Gestion'].sum() - temp_ble['Dispo'].sum()) ) 
print ('\nL expression est verifié pour le blé de France.')
print ('\n\nLes infomations liées au blé de france :')
print (temp_ble.transpose())

Pour le Blé : 

gestion - dispo = 0.0

dispo - utilisation = 0.0

L expression est verifié pour le blé de France.


Les infomations liées au blé de france :
                                                       4958
Zone                                                 France
Code zone                                                68
Année                                                  2013
origine                                            vegetaux
Code Produit                                           2511
Produit                                                 Blé
Aliments pour animaux (Milliers de tonnes)             7822
Utilisations non alimentaire (Milliers de tonnes)      2824
Disponibilité alimentaire (Kcal/personne/jour)          811
Disponibilité alimentaire (kg/personne/an)           108.43
Disponibilité de matière grasse (g/personne/jour)       4.2
Disponibilité de protéines (g/personne/jour)          25.99
Disponibilité intérieure (Milliers de tonnes)         20298
Exp

# Question 3 : 
Calculez (pour chaque pays et chaque produit) 
    - la disponibilité alimentaire en kcal 
    - puis en kg de protéines.

In [6]:
#dispo en Kcal/personne/jour * nbre personnes * 365 jours
df['Disponibilité (Kcal)'] = df['Disponibilité alimentaire (Kcal/personne/jour)'] * df ['Population'] * 365

#dispo en Kg/personne/jour * nbre personnes * 365 jours
df['Disponibilité (Kg)'] = df['Disponibilité de protéines (g/personne/jour)'] * df ['Population'] * 365/1000

In [7]:
df [df['origine'] == 'vegetaux']  [['Zone','Produit','Disponibilité alimentaire (Kcal/personne/jour)', 'Population','Disponibilité (Kcal)'] ].head()

,Zone,Produit,Disponibilité alimentaire (Kcal/personne/jour),Population,Disponibilité (Kcal)
11,Afghanistan,Blé,1369.0,30552000,1.526638e+13
12,Afghanistan,Orge,26.0,30552000,2.899385e+11
13,Afghanistan,Maïs,21.0,30552000,2.341811e+11
14,Afghanistan,Millet,3.0,30552000,3.345444e+10
15,Afghanistan,"Céréales, Autres",0.0,30552000,0.000000e+00


In [8]:
df[df['origine'] == 'vegetaux'] [['Zone','Produit','Disponibilité alimentaire (kg/personne/an)' , 'Population','Disponibilité (Kg)'  ] ].head()

,Zone,Produit,Disponibilité alimentaire (kg/personne/an),Population,Disponibilité (Kg)
11,Afghanistan,Blé,160.23,30552000,411601126.8
12,Afghanistan,Orge,2.92,30552000,8809669.2
13,Afghanistan,Maïs,2.50,30552000,6244828.8
14,Afghanistan,Millet,0.40,30552000,892118.4
15,Afghanistan,"Céréales, Autres",0.00,30552000,0.0


# Question 4 : 

A partir de ces dernières informations, et à partir du poids de la disponibilité alimentaire (pour chaque pays et chaque produit), calculez pour chaque produit le ratio "énergie/poids", que vous donnerez en kcal/kg. Vous pouvez vérifier la cohérence de votre calcul en comparant ce ratio aux données disponibles sur internet, par exemple en cherchant la valeur calorique d'un oeuf.

In [9]:
#suppr des valeurs nan
df['Disponibilité (Kg)'] = df['Disponibilité (Kg)'].fillna(0)
df['Disponibilité (Kcal)'] = df['Disponibilité (Kcal)'].fillna(0)
df['Nourriture (Milliers de tonnes)'] = df['Nourriture (Milliers de tonnes)'].fillna(0)

#Creation d un colonne correspondant au ratio "énergie/poids"
df['ratio kcal/kg'] = df.loc [df['Nourriture (Milliers de tonnes)'] > 0] ['Disponibilité (Kcal)'] / df['Nourriture (Milliers de tonnes)'].apply(lambda x : x * 1000000)
df['ratio kcal/kg'] = df['ratio kcal/kg'].fillna(0)
df[df['origine'] == 'vegetaux'][[ 'Zone' , 'Produit' , 'ratio kcal/kg'] ].head()

,Zone,Produit,ratio kcal/kg
11,Afghanistan,Blé,3118.769381
12,Afghanistan,Orge,3257.735730
13,Afghanistan,Maïs,3081.330000
14,Afghanistan,Millet,2787.870000
15,Afghanistan,"Céréales, Autres",0.000000


In [10]:
#group by par produit
print ('Voici le ratio "énergie/poids" par produit :')
df[df['ratio kcal/kg']  > 0 ]  [['Produit','ratio kcal/kg' ]].groupby('Produit').agg('mean').sort_values (by = 'ratio kcal/kg', ascending = False).iloc[:20]

Voici le ratio "énergie/poids" par produit :


,ratio kcal/kg
Produit,
Huiles de Foie de Poisso,9982.932500
Huile de Sésame,9433.744729
Huile de Son de Riz,9361.549883
Huiles de Poissons,9003.715504
Huile d'Arachide,8935.765811
Huile d'Olive,8931.223447
Huile de Germe de Maïs,8871.762566
Huil Plantes Oleif Autr,8848.222893
Huile de Palmistes,8789.540277


In [11]:
#la valeur calorique d'un oeuf
val_oeuf = df[( df.loc[:,'Produit'] == 'Oeufs' ) ]  ['ratio kcal/kg'].mean()
val_oeuf_fr = df[( df.loc[:,'Produit'] == 'Oeufs')  & (df ['Zone'] == 'France'  ) ]  ['ratio kcal/kg']

print ('''la valeur calorique d un oeuf :
       - est de {} en kcal/kg en france 
       - est de {} en kcal/kg en moyenne dans le monde  
\nLa valeur sur le net est autour de 1450 kcal/kg. '''.format ( int  (val_oeuf_fr) ,    int (val_oeuf) ))

la valeur calorique d un oeuf :
       - est de 1417 en kcal/kg en france 
       - est de 1306 en kcal/kg en moyenne dans le monde  

La valeur sur le net est autour de 1450 kcal/kg. 


# Question 5 : 
Citez 5 aliments parmi les 20 aliments les plus caloriques, en utilisant le ratio énergie/poids.
Étonnamment, il arrive que ce ratio soit différent en fonction du pays. Il faudra donc réaliser pour chaque aliment une moyenne sur les différents pays. Vous créerez donc une nouvelle table grâce à une agrégation. Attention à bien retirer les valeurs égales à 0 afin de ne pas fausser le calcul de la moyenne.
Citez 5 aliments parmi les 20 aliments les plus riches en protéines.

In [12]:
#pour le ratio Kcal / kg
df[df['ratio kcal/kg']  > 0 ]  [['Produit','ratio kcal/kg' ]].groupby('Produit').agg('mean').sort_values (by = 'ratio kcal/kg', ascending = False).head()

,ratio kcal/kg
Produit,
Huiles de Foie de Poisso,9982.932500
Huile de Sésame,9433.744729
Huile de Son de Riz,9361.549883
Huiles de Poissons,9003.715504
Huile d'Arachide,8935.765811


In [13]:
#calcul du % de proteine dans la nourriture
df['prot_weight(%)'] = df [df['Nourriture (Milliers de tonnes)'] >0] ['Disponibilité (Kg)'] * 100 / df['Nourriture (Milliers de tonnes)'].apply (lambda x: x*1000000)
df[df['prot_weight(%)']  > 0 ] [['Produit','prot_weight(%)' ]].groupby('Produit').agg('mean').sort_values (by = 'prot_weight(%)', ascending = False).head()

,prot_weight(%)
Produit,
Graines Colza/Moutarde,28.536511
Soja,28.533053
Arachides Decortiquees,25.146613
Pois,22.324375
Légumineuses Autres,21.971635


# Question 6 : 

Calculez, pour les produits végétaux uniquement, la disponibilité intérieure mondiale exprimée en kcal.


In [14]:
df[['Disponibilité intérieure (Milliers de tonnes)' , 'Dispo']]

,Disponibilité intérieure (Milliers de tonnes),Dispo
0,140.0,140.0
1,150.0,150.0
2,75.0,75.0
3,12.0,12.0
4,53.0,53.0
...,...,...
15697,0.0,0.0
15698,0.0,0.0
15699,0.0,0.0
15700,49.0,49.0


In [15]:
# pour les vegetaux slt : la dispo en millier de tonne * 10000000 pour avoir des kg puis utiliser le ratio 

#calories
df['Disponibilité vegetal (Kcal)'] = df[df['origine'] == 'vegetaux'] [ 'Disponibilité intérieure (Milliers de tonnes)' ] * 1000000 * df['ratio kcal/kg']

som_kcal = df[df['origine'] == 'vegetaux'] ['Disponibilité vegetal (Kcal)'].sum()

print ('''La Disponibilité vegetal totale est de {} Kcal

Voici les 1er éléments pour la disponibilité vegetale : '''.format (som_kcal))


df[[ 'Zone' , 'Produit' , 'Disponibilité vegetal (Kcal)']].dropna().head()

La Disponibilité vegetal totale est de 1.23127211628784e+16 Kcal

Voici les 1er éléments pour la disponibilité vegetale : 


,Zone,Produit,Disponibilité vegetal (Kcal)
11,Afghanistan,Blé,1.868767e+13
12,Afghanistan,Orge,1.707054e+12
13,Afghanistan,Maïs,9.644563e+11
14,Afghanistan,Millet,3.624231e+10
15,Afghanistan,"Céréales, Autres",0.000000e+00


In [16]:
# pour les vegetaux slt : la dispo en millier de tonne * 10000000 pour avoir des kg 
#proteines
# df['Disponibilité vegetal (Kg)'] = df[df['origine'] == 'vegetaux']['Disponibilité de protéines (g/personne/jour)'] * df['Population'] * 365 * df['prot_weight(%)'].apply ( lambda x : x  / 100 )  

# som_kg = df[df['origine'] == 'vegetaux'] ['Disponibilité vegetal (Kg)'].sum()

# print ('Disponibilité vegetal (Kg) :', som_kg)

# Question 7 : 
Combien d'humains pourraient être nourris si toute la disponibilité intérieure mondiale de produits végétaux était utilisée pour de la nourriture ? Donnez les résultats en termes de calories, puis de protéines, et exprimez ensuite ces 2 résultats en pourcentage de la population mondiale.

In [17]:
# le besoin en calories

#pourcentage
def p (x , total = pop_monde) : 
    return x * 100 / total

#le besoin 
besoin_energ_an = 2400*365  # le besoin énergétique en kcal/personne/jour pour 1 an 

#le calcul = 
x = som_kcal / besoin_energ_an

print ('Il y a {:.2f} personnes qui peuvent etre nourries soit {:.2f} % de la population modiale'.format ( x.sum() , p ( x ) )) 

Il y a 14055617765.84 personnes qui peuvent etre nourries soit 200.87 % de la population modiale


In [18]:
# le besoin en proteine

#le besoin
besoin_prot_an = 0.88 * 70 *365* 0.001  # le besoin en proteine 0.9 g / kg / jours  1 humain 70kg
# la dispo en prot
som_kg = df[df['prot_weight(%)']  > 0.1 ] ['Disponibilité intérieure (Milliers de tonnes)'].apply ( lambda x : x  * 1000000 ) * df['prot_weight(%)'].apply ( lambda x : x  / 100 ) 
#calcul
x = som_kg.apply ( lambda x : x / besoin_prot_an ) 
x = x.sum()

print ('Il y a {:.2f} personnes qui peuvent etre nourries soit {:.2f} % de la population modiale'.format ( x , p (x , pop_monde) ) )

Il y a 16956616186.13 personnes qui peuvent etre nourries soit 242.33 % de la population modiale


# Question 8 : 
Combien d'humains pourraient être nourris si toute la disponibilité alimentaire en produits végétaux la nourriture végétale destinée aux animaux et les pertes de produits végétaux étaient utilisés pour de la nourriture ? Donnez les résultats en termes de calories, puis de protéines, et exprimez ensuite ces 2 résultats en pourcentage de la population mondiale.

In [19]:
#creation d un df pour faciliter 
df_vegie = df[df['origine'] == 'vegetaux'] [[ 'Zone', 'Produit' , 'Nourriture (Milliers de tonnes)' , 'Disponibilité intérieure (Milliers de tonnes)' , 'Population' , 'Aliments pour animaux (Milliers de tonnes)' , 'Utilisations non alimentaire (Milliers de tonnes)' , 'ratio kcal/kg' , 'prot_weight(%)' ]]

df_vegie.head(1).transpose()

,11
Zone,Afghanistan
Produit,Blé
Nourriture (Milliers de tonnes),4895
Disponibilité intérieure (Milliers de tonnes),5992
Population,30552000
Aliments pour animaux (Milliers de tonnes),0
Utilisations non alimentaire (Milliers de tonnes),0
ratio kcal/kg,3118.77
prot_weight(%),8.4086


In [20]:
#Calories :

#creation d une colonne somme
df_vegie['som (Milliers de tonnes)'] = df ['Nourriture (Milliers de tonnes)'] + df ['Aliments pour animaux (Milliers de tonnes)'] + df ['Utilisations non alimentaire (Milliers de tonnes)'] 
#creation d une colonne somme pour les calories
df_vegie['som cal'] = df_vegie['som (Milliers de tonnes)'] * 1000000 * df [ 'ratio kcal/kg' ]
# besoin_energ_an = 2700*365     # le besoin

#calcul
x = df_vegie['som cal'].sum() / besoin_energ_an 

print ('Il y a {:.0f} personnes qui peuvent etre nourries soit {:.1f} % de la population modiale'.format ( x.sum() ,p(x ) )) 

Il y a 11694024316 personnes qui peuvent etre nourries soit 167.1 % de la population modiale


In [21]:
# Proteines : 
#creation d une colonne pour les proteines
df_vegie['som prot'] = df_vegie [ df_vegie ['prot_weight(%)']  > 0.1 ] ['som (Milliers de tonnes)'] * 1000000 * df['prot_weight(%)'].apply ( lambda x : x  / 100 )

#calcul
x = (df_vegie['som prot'].apply ( lambda x : x / besoin_prot_an ) ).sum()

print ('Il y a {:.2f} personnes qui peuvent etre nourries soit {:.2f} % de la population modiale'.format ( x , p (x ) ) )

Il y a 9197154368.55 personnes qui peuvent etre nourries soit 131.44 % de la population modiale


# Question 9 : 

Combien d'humains pourraient être nourris avec la disponibilité alimentaire mondiale ? Donnez les résultats en termes de calories, puis de protéines, et exprimez ensuite ces 2 résultats en pourcentage de la population mondiale.

In [22]:
#Calcul calorie
df_dispo_cal = df['Nourriture (Milliers de tonnes)'].apply (lambda x : x  * 1000000 ) * df [ 'ratio kcal/kg' ]
# besoin_energ_an = 2500*365     # le besoin
x = df_dispo_cal.sum() / besoin_energ_an 

#Calcul Proteine
df_dispo_prot = df[df['prot_weight(%)']  > 0.1 ] ['Nourriture (Milliers de tonnes)'].apply ( lambda x : x  * 1000000 ) * df['prot_weight(%)'].apply ( lambda x : x  / 100 ) 
# besoin_prot_an = 0.9 * 66 *365* 0.001  # le besoin
y = df_dispo_prot.apply ( lambda x : x / besoin_prot_an ) .sum()

print ('''La disponibilité alimentaire permettrait de nourrir :
     - En terme de calories :  {:.0f} personnes soit {:.1f} % de la population modiale;
     - En terme de proteines : {:.0f} personnes soit {:.1f} % de la population modiale;
     ''' .format ( x.sum() ,p(x ,pop_monde) , y , p (y , pop_monde) )  ) 



La disponibilité alimentaire permettrait de nourrir :
     - En terme de calories :  8403343785 personnes soit 120.1 % de la population modiale;
     - En terme de proteines : 9212939188 personnes soit 131.7 % de la population modiale;
     


# Question 10 : 
A partir des données téléchargées qui concernent la sous-nutrition, répondez à cette question : Quelle proportion de la population mondiale est considérée comme étant en sous-nutrition ?
Établissez la liste des produits (ainsi que leur code) considéré comme des céréales selon la FAO.

Repérez dans vos données les informations concernant les céréales (par exemple en créant une colonne de type booléen nommée "is_cereal").

In [23]:
#population en sous alimentation
som_ss_alim = df[['Zone','Pop sous alimente']].drop_duplicates()  ['Pop sous alimente'].sum()
p_ss_alim = df[['Zone','Pop sous alimente']].drop_duplicates()  ['Pop sous alimente'].sum() / pop_monde

print ('Le nombre de personne en sous nutrition est {} soit {} % de la pop mondiale :'.format (som_ss_alim , p (som_ss_alim , pop_monde) ) )

Le nombre de personne en sous nutrition est 746000000.0 soit 10.661215441441488 % de la pop mondiale :


In [24]:
#creation dune colonne 
df['ratio ss alim'] = df ['Pop sous alimente'] /  df ['Population']

print ( 'Les pays avec le ratio population sous alimenté : ' )
df.groupby ('Zone').agg( {'ratio ss alim' : 'mean'} ).sort_values('ratio ss alim', ascending = False).iloc [1:10] 

Les pays avec le ratio population sous alimenté : 


,ratio ss alim
Zone,
Kiribati,0.980392
Saint-Vincent-et-les Grenadines,0.917431
Samoa,0.526316
Sao Tomé-et-Principe,0.518135
Haïti,0.504022
Zambie,0.481464
Zimbabwe,0.466431
République centrafricaine,0.433276
République populaire démocratique de Corée,0.425788


In [25]:
#Établissez la liste des produits

#creation col
df['is_cereal'] = 0  

#creation d un liste cereale 
liste_cereal = df_cereales ['Code Produit'].unique ()

#si code produit correspond a un code de la liste cereal 
df.loc [df ['Code Produit'].isin (liste_cereal),'is_cereal'] = 1     #prend en compte tt la table
#df.loc [df ['origine'] == 'cereales','is_cereal'] = 1   #prend en compte slt les element de la table cereales

print ('la liste des produits sont :')
liste_cereal

la liste des produits sont :


array([2511, 2805, 2513, 2514, 2517, 2515, 2516, 2518, 2520], dtype=int64)

# Question 11 : 

En ne prenant en compte que les céréales destinées à l'alimentation (humaine et animale), quelle proportion (en termes de poids) est destinée à l'alimentation animale ?


In [26]:
#Aliments pour animaux (Milliers de tonnes)
alim_anim = df.loc[df ['is_cereal'] == 1 , 'Aliments pour animaux (Milliers de tonnes)'].sum()

#Aliments pour Nourriture (Milliers de tonnes)
alim_hum = df.loc[df ['is_cereal'] == 1 , 'Nourriture (Milliers de tonnes)'].sum()

x = alim_anim / (alim_anim + alim_hum) * 100

print ('Il y a {:.2f} % des cereale destinée a l alimentation des animaux'.format ( x ) ) 

Il y a 45.91 % des cereale destinée a l alimentation des animaux


# Question 12 : 
Donnez les 3 produits qui on t la plus grande valeur pour chacun des 2 ratios (vous aurez donc 6 produits à citer) + questions complementaires

In [27]:
#Repérez les 15 produits les plus exportés par ce groupe de pays.

df[df['Pop sous alimente'] >1.0].groupby ('Produit').agg('sum')  ['Exportations (Milliers de tonnes)'].sort_values(ascending = False).head(15)

Produit
Huile de Palme         46324.0
Maïs                   37906.0
Manioc                 35851.0
Riz (Eq Blanchi)       33093.0
Sucre Eq Brut          26098.0
Blé                    25342.0
Légumes, Autres        20139.0
Bananes                17988.0
Lait - Excl Beurre     14973.0
Fruits, Autres         14690.0
Soja                   14391.0
Poissons Pelagiques    14181.0
Tomates                 8935.0
Pommes                  7897.0
Oranges, Mandarines     7106.0
Name: Exportations (Milliers de tonnes), dtype: float64

In [28]:
# Parmi les données des bilans alimentaires au niveau mondial, sélectionnez les 200 plus grandes importations de ces produits (1 importation = une quantité d'un produit donné importée par un pays donné)
# Groupez ces importations par produit, afin d'avoir une table contenant 1 ligne pour chacun des 15 produits. Ensuite, calculez pour chaque produit les 2 quantités suivantes :

df[df['Pop sous alimente'] >0.1].groupby ('Produit').agg('sum')  ['Importations (Milliers de tonnes)'].sort_values(ascending = False).head(15)

Produit
Blé                    104658.0
Soja                    78186.0
Maïs                    61836.0
Lait - Excl Beurre      43467.0
Sucre Eq Brut           39288.0
Manioc                  34492.0
Huile de Palme          31833.0
Riz (Eq Blanchi)        28494.0
Orge                    21590.0
Poissons Pelagiques     15961.0
Fruits, Autres           8628.0
Huile de Soja            8202.0
Viande de Volailles      7054.0
Pommes de Terre          6788.0
Légumes, Autres          6400.0
Name: Importations (Milliers de tonnes), dtype: float64

In [29]:
# le ratio entre la quantité destinés aux "Autres utilisations" (Other uses) et la disponibilité intérieure.
df_temp = df [(df['Disponibilité intérieure (Milliers de tonnes)'] > 1) & (df['Utilisations non alimentaire (Milliers de tonnes)'] > 0)  ] [ [ 'Zone' , 'Produit' ,'Utilisations non alimentaire (Milliers de tonnes)','Disponibilité intérieure (Milliers de tonnes)'] ]
df_temp['Ratio'] = df_temp['Utilisations non alimentaire (Milliers de tonnes)'] / df_temp['Disponibilité intérieure (Milliers de tonnes)'] 
print ('les ratios "Autres utilisations / disponibilité intérieure" les plus elevés : ')
df_temp.groupby('Produit').agg('sum').sort_values('Ratio', ascending = False)['Ratio'] .head(3) 

les ratios "Autres utilisations / disponibilité intérieure" les plus elevés : 


Produit
Alcool, non Comestible     97.000000
Huile de Palme             89.572989
Huil Plantes Oleif Autr    84.142199
Name: Ratio, dtype: float64

In [30]:
#Aliments pour animaux (Milliers de tonnes)
temp = df.loc[df ['is_cereal'] == 1  , ['Zone','origine','Code Produit','Produit','Aliments pour animaux (Milliers de tonnes)', 'Nourriture (Milliers de tonnes)']].dropna()

#creation des colonnes  
temp ['ratio animal'] = temp ['Aliments pour animaux (Milliers de tonnes)'] /  ( temp ['Aliments pour animaux (Milliers de tonnes)'] + temp ['Nourriture (Milliers de tonnes)']  )
#pour le ratio animal
print ('les ratios "animal" les plus elevés : ')
temp[(temp ['Aliments pour animaux (Milliers de tonnes)'] > 0) & (df ['Nourriture (Milliers de tonnes)'] > 0)].sort_values ( 'ratio animal' , ascending = False).head(3)

les ratios "animal" les plus elevés : 


<ipython-input-30-d017fda3cc06>:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  temp[(temp ['Aliments pour animaux (Milliers de tonnes)'] > 0) & (df ['Nourriture (Milliers de tonnes)'] > 0)].sort_values ( 'ratio animal' , ascending = False).head(3)


,Zone,origine,Code Produit,Produit,Aliments pour animaux (Milliers de tonnes),Nourriture (Milliers de tonnes),ratio animal
6215,Hongrie,vegetaux,2514,Maïs,2099.0,3.0,0.998573
6214,Hongrie,vegetaux,2513,Orge,475.0,1.0,0.997899
4493,Espagne,vegetaux,2513,Orge,8000.0,17.0,0.997880


# Question 13 : 
Combien de tonnes de céréales pourraient être libérées si les USA diminuaient leur production de produits animaux de 10% ?

In [31]:
df['Tonne libéré (10%)'] = df[(df.loc [:,'Zone'] == "États-Unis d'Amérique") & (df.loc [:,'Code Produit'] .isin (liste_cereal))  & (df.loc [:,'Aliments pour animaux (Milliers de tonnes)'] > 0) ] ['Aliments pour animaux (Milliers de tonnes)']*0.1 
df [df['Tonne libéré (10%)'] >0]   [['Produit' ,'Aliments pour animaux (Milliers de tonnes)' , 'Tonne libéré (10%)' ]]

print ('Si les USA diminuaient leur production de produits animaux de 10% : {:.2f} milles tonnes de Cereales seraient libérées'.format ( df['Tonne libéré (10%)'].sum() / 1000)  )

Si les USA diminuaient leur production de produits animaux de 10% : 14.01 milles tonnes de Cereales seraient libérées


# Question 14 : 
En Thaïlande, quelle proportion de manioc est exportée ? Quelle est la proportion de personnes en sous-nutrition?

In [32]:
# La proportion de manioc exporté

magnioc = df[ (df.loc [:, 'Zone'] == 'Thaïlande') & (df.loc[:, 'Produit'] == 'Manioc' ) ] ['Exportations (Milliers de tonnes)'].sum ()

som_export = df[ df.loc [:, 'Zone'] == 'Thaïlande']['Exportations (Milliers de tonnes)'].sum()
 
print ('La quantité exporté de magnioc (Millier de tonnes) : {} qui correspond à {}% des exportations alim.\nLa totalité des exportations (Millier de tonnes): {}  '.format ( magnioc , (magnioc / som_export * 100).round(),som_export ))

La quantité exporté de magnioc (Millier de tonnes) : 25214.0 qui correspond à 50.0% des exportations alim.
La totalité des exportations (Millier de tonnes): 50430.0  


In [33]:
#La population en sous nutrition
thai_pop_ss_alim = df [df.loc [ : ,'Zone'] == 'Thaïlande'] ['Pop sous alimente'].unique() [0]

print ('Nombre de personnes sous alimentés : {} qui correspond à {:.2f}% de la population de la thailande'.format ( thai_pop_ss_alim , thai_pop_ss_alim *100 / df[ (df.loc [:, 'Zone'] == 'Thaïlande') ]['Population'].mean()))

Nombre de personnes sous alimentés : 5600000.0 qui correspond à 8.36% de la population de la thailande


# Question 15 : 

Votre base devra contenir ces différentes tables :

Une table appelée population, contenant la population de chaque pays pour 2013. Elle devra contenir 4 colonnes : pays, code_pays, annee, population.

Proposez une clé primaire pertinente pour cette table.

In [34]:
import sqlite3, sqlalchemy
from sqlalchemy import Table, Column, Integer, String, ForeignKey, MetaData, create_engine, text, inspect
from IPython.display import Markdown, display

engine = create_engine('sqlite:///food.db', echo=True)

meta = MetaData()

In [35]:
#filtrer d un df 
df_pop_s = df_pop[['Code zone' , 'Zone' , 'Année' , 'Valeur' ]]

#creation de la cle
#df_pop_s ['key']  = df_pop['Année'].apply (lambda x : str (x)) + '_' +  df_pop['Code zone' ].apply (lambda x : str (x))

#rename
df_pop_s.rename (columns = { 'Code zone' : 'code_pays' , 
                            'Zone'  :  'pays' , 
                            'Année' : 'annee' ,
                            'Valeur' : 'population'} , 
                 inplace = True)

df_pop_s['population'] = df_pop_s['population'].apply (lambda x :x * 1000)
df_pop_s.set_index ( 'code_pays' , inplace = True)

# creation d une table
df_pop_s.to_sql (
    name = 'temp',
    if_exists = 'replace',
    con = engine ,
    index = True , 
    
)

#ce sera une clé composé comprenant le code année et le code pays 

connection = engine.raw_connection()
c = connection.cursor()

#ALTER TABLE Population RENAME TO popTest;
c.executescript('''
    PRAGMA foreign_keys=off;

    BEGIN TRANSACTION;    
    drop table if exists population;  
    CREATE TABLE population (code_pays TEXT ,
                            pays TEXT,
                            annee TEXT ,
                            population TEXT,
                            PRIMARY KEY ("code_pays", "annee")
                            );

   INSERT INTO population SELECT * FROM temp;    
   DROP TABLE temp;
   COMMIT TRANSACTION; 
   PRAGMA foreign_keys=on;''')

#close out the connection
c.close()
connection.close()

#Vérification de notre commit
with engine.connect() as conn:
    statement = text("select * from population;")
    result = conn.execute(statement)
    print (result.fetchall())

2020-12-21 13:58:13,401 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2020-12-21 13:58:13,403 INFO sqlalchemy.engine.base.Engine ()
2020-12-21 13:58:13,405 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2020-12-21 13:58:13,406 INFO sqlalchemy.engine.base.Engine ()
2020-12-21 13:58:13,407 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("temp")
2020-12-21 13:58:13,408 INFO sqlalchemy.engine.base.Engine ()
2020-12-21 13:58:13,410 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("temp")
2020-12-21 13:58:13,411 INFO sqlalchemy.engine.base.Engine ()
2020-12-21 13:58:13,415 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE "temp" (
	code_pays BIGINT, 
	pays TEXT, 
	annee BIGINT, 
	population BIGINT
)


2020-12-21 13:58:13,416 INFO sqlalchemy.engine.base.Engine ()
2020-12-21 13:58:13,424 INFO sqlalchemy.engine.base.Engine COMMIT
2020-12-21 13:58:13,426 INFO sqlalchemy.engine.base.

C:\Users\bilel\Anaconda3\lib\site-packages\pandas\core\frame.py:4296: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(
<ipython-input-35-88d4a3e1a574>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pop_s['population'] = df_pop_s['population'].apply (lambda x :x * 1000)


# Question 16 : 

Une table appelée dispo_alim contenant pour chaque pays et pour chaque produit en 2013, les informations suivantes:
la nature du produit (deux valeurs possibles : “animal” ou “végétal”)
disponibilité alimentaire en tonnes
disponibilité alimentaire en Kcal/personne/jour
disponibilité alimentaire de protéines en g/personne/jour
disponibilité alimentaire de matières grasses en g/personne/jour
Elle devra contenir ces colonnes : pays, code_pays, année, produit, code_produit, origin, dispo_alim_tonnes, dispo_alim_kcal_p_j, dispo_prot, dispo_mat_gr .

Proposez une clé primaire pertinente pour cette table.

In [36]:
#creation d un df
df_s = df[df ['origine'] != 'cereales'][['Code zone' , 'Zone' , 'Année' , 'Produit' , 'Code Produit' , 'origine' ,
           'Disponibilité intérieure (Milliers de tonnes)',
           'Disponibilité alimentaire (Kcal/personne/jour)',
           'Disponibilité alimentaire (kg/personne/an)',
           'Disponibilité de protéines (g/personne/jour)'               ]]

#renamme
df_s.rename (columns = { 'Code zone' : 'code_pays' , 
                        'Zone'  :  'pays' , 
                       'Année' : 'annee' , 
                        'Produit' : 'produit' , 
                        'Code Produit' : 'code_produit' ,
                        'origine' : 'origin' , 
                        'Disponibilité intérieure (Milliers de tonnes)' : 'dispo_alim_tonnes' ,
                        'Disponibilité alimentaire (Kcal/personne/jour)'  : 'dispo_alim_kcal_p_j' ,
                        'Disponibilité alimentaire (kg/personne/an)'    : 'dispo_prot'     ,
                        'Disponibilité de protéines (g/personne/jour)' : 'dispo_mat_gr'    
                       } , inplace = True)

df_s.set_index ( 'code_pays' , inplace = True)

# creation d une table
df_s.to_sql (
    name = 'temp1',
    if_exists = 'replace',
    con = engine ,

)

#on utilise une cle composee avec le code annee le code pays et le code produit

connection = engine.raw_connection()
c = connection.cursor()

#ALTER TABLE Population RENAME TO popTest;
c.executescript('''
    PRAGMA foreign_keys=off;

    BEGIN TRANSACTION;    
    drop table if exists dispo_alim;  
    CREATE TABLE dispo_alim ( 'code_pays' BIGINT,
                            'pays' TEXT,
                            'annee' BIGINT,
                            'produit' TEXT,
                            'code_produit' BIGINT,
                            'origin' TEXT,
                            'dispo_alim_tonnes' FLOAT,
                            'dispo_alim_kcal_p_j' FLOAT,
                            'dispo_prot' FLOAT,
                            'dispo_mat_gr' FLOAT,
                            PRIMARY KEY ("code_pays", "code_produit" , "annee")    
                            );

   INSERT INTO dispo_alim SELECT * FROM temp1;    
   DROP TABLE temp1;
   COMMIT TRANSACTION; 
   PRAGMA foreign_keys=on;''')

#close out the connection
c.close()
connection.close()

#Vérification de notre commit
with engine.connect() as conn:
    statement = text("select * from dispo_alim;")
    result = conn.execute(statement)
    print (result.fetchall())



2020-12-21 13:58:13,500 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("temp1")
2020-12-21 13:58:13,501 INFO sqlalchemy.engine.base.Engine ()
2020-12-21 13:58:13,502 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("temp1")
2020-12-21 13:58:13,504 INFO sqlalchemy.engine.base.Engine ()
2020-12-21 13:58:13,508 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE temp1 (
	code_pays BIGINT, 
	pays TEXT, 
	annee BIGINT, 
	produit TEXT, 
	code_produit BIGINT, 
	origin TEXT, 
	dispo_alim_tonnes FLOAT, 
	dispo_alim_kcal_p_j FLOAT, 
	dispo_prot FLOAT, 
	dispo_mat_gr FLOAT
)


2020-12-21 13:58:13,509 INFO sqlalchemy.engine.base.Engine ()
2020-12-21 13:58:13,516 INFO sqlalchemy.engine.base.Engine COMMIT
2020-12-21 13:58:13,517 INFO sqlalchemy.engine.base.Engine CREATE INDEX ix_temp1_code_pays ON temp1 (code_pays)
2020-12-21 13:58:13,518 INFO sqlalchemy.engine.base.Engine ()
2020-12-21 13:58:13,526 INFO sqlalchemy.engine.base.Engine COMMIT
2020-12-21 13:58:13,537 INFO sqlalchemy.en

[(2, 'Afghanistan', 2013, 'Viande de Bovins', 2731, 'animaux', 140.0, 27.0, 4.59, 1.89), (2, 'Afghanistan', 2013, "Viande d'Ovins/Caprins", 2732, 'animaux', 150.0, 31.0, 4.92, 1.91), (2, 'Afghanistan', 2013, 'Viande de Volailles', 2734, 'animaux', 75.0, 9.0, 2.45, 0.87), (2, 'Afghanistan', 2013, 'Viande, Autre', 2735, 'animaux', 12.0, 1.0, 0.38, 0.17), (2, 'Afghanistan', 2013, 'Abats Comestible', 2736, 'animaux', 53.0, 5.0, 1.72, 0.77), (2, 'Afghanistan', 2013, 'Graisses Animales Crue', 2737, 'animaux', 15.0, 12.0, 0.49, 0.01), (2, 'Afghanistan', 2013, 'Beurre, Ghee', 2740, 'animaux', 36.0, 23.0, 1.17, 0.03), (2, 'Afghanistan', 2013, 'Crème', 2743, 'animaux', 18.0, 3.0, 0.6, 0.04), (2, 'Afghanistan', 2013, 'Oeufs', 2744, 'animaux', 47.0, 5.0, 1.3, 0.38), (2, 'Afghanistan', 2013, 'Poissons Eau Douce', 2761, 'animaux', 2.0, 0.0, 0.07, 0.02), (2, 'Afghanistan', 2013, 'Lait - Excl Beurre', 2848, 'animaux', 2083.0, 100.0, 62.23, 6.12), (2, 'Afghanistan', 2013, 'Blé', 2511, 'vegetaux', 5992.

# Question 17 :

Une table appelée equilibre_prod contenant pour chaque pays et pour chaque produit en 2013, les quantités suivantes :
disponibilité intérieure
aliments pour animaux
semences
pertes
transformés
nourriture
autres utilisations
Elle devra contenir ces colonnes : pays, code_pays, année, produit, code_produit, dispo_int, alim_ani, semences, pertes, transfo, nourriture, autres_utilisations.

Proposez une clé primaire pertinente pour cette table.

In [37]:
#creation d un df
df_s = df[df ['origine'] != 'cereales'][['Zone' , 'Code zone' , 'Année' , 
           'Produit' , 'Code Produit' , 
           'Disponibilité intérieure (Milliers de tonnes)' , 
           'Aliments pour animaux (Milliers de tonnes)' , 
           'Semences (Milliers de tonnes)' , 'Pertes (Milliers de tonnes)' , 
           'Traitement (Milliers de tonnes)' ,
           'Nourriture (Milliers de tonnes)' , 
           'Utilisations non alimentaire (Milliers de tonnes)']]

#renamme
df_s.rename (columns = { 
        'Code zone' : 'code_pays' , 
    'Zone'  : 'pays', 
    'Année'  : 'annee',            
    'Produit'  : 'produit', 
    'Code Produit' : 'code_produit' ,           
    'Disponibilité intérieure (Milliers de tonnes)'  : 'dispo_int',            
    'Aliments pour animaux (Milliers de tonnes)'  : 'alim_ani',            
    'Semences (Milliers de tonnes)'  : 'semences', 
    'Pertes (Milliers de tonnes)'  : 'pertes',            
    'Traitement (Milliers de tonnes)'  : 'transfo',           
    'Nourriture (Milliers de tonnes)'  : 'nourriture',            
    'Utilisations non alimentaire (Milliers de tonnes)' : 'autres_utilisations' 
} , inplace = True)


df_s.set_index ( 'code_pays' , inplace = True)


# creation d une table
df_s.to_sql (
    name = 'temp2',
    if_exists = 'replace',
    con = engine ,
    index = True )

#on utilise une cle composee avec le code annee le code pays et le code produit



connection = engine.raw_connection()
c = connection.cursor()

#ALTER TABLE Population RENAME TO popTest;
c.executescript('''
    PRAGMA foreign_keys=off;

    BEGIN TRANSACTION;    
    drop table if exists equilibre_prod;  
    CREATE TABLE equilibre_prod ( 'code_pays' BIGINT,
                            'pays' TEXT,
                            'annee' BIGINT,
                            'produit' TEXT,
                            'code_produit' BIGINT,                  
                            'dispo_int' FLOAT,
                            'alim_ani' FLOAT,
                            'semences' FLOAT,
                            'pertes' FLOAT,
                            'transfo' FLOAT,
                            'nourriture' FLOAT,
                            'autres_utilisations' FLOAT,
                            PRIMARY KEY ("code_pays", "code_produit" , "annee")                          
                            );

   INSERT INTO equilibre_prod SELECT * FROM temp2;    
   DROP TABLE temp2;
   COMMIT TRANSACTION; 
   PRAGMA foreign_keys=on;''')

#close out the connection
c.close()
connection.close()

#Vérification de notre commit
with engine.connect() as conn:
    statement = text("select * from equilibre_prod;")
    result = conn.execute(statement)
    print (result.fetchall())

2020-12-21 13:58:14,184 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("temp2")
2020-12-21 13:58:14,185 INFO sqlalchemy.engine.base.Engine ()
2020-12-21 13:58:14,187 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("temp2")
2020-12-21 13:58:14,188 INFO sqlalchemy.engine.base.Engine ()
2020-12-21 13:58:14,191 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE temp2 (
	code_pays BIGINT, 
	pays TEXT, 
	annee BIGINT, 
	produit TEXT, 
	code_produit BIGINT, 
	dispo_int FLOAT, 
	alim_ani FLOAT, 
	semences FLOAT, 
	pertes FLOAT, 
	transfo FLOAT, 
	nourriture FLOAT, 
	autres_utilisations FLOAT
)


2020-12-21 13:58:14,192 INFO sqlalchemy.engine.base.Engine ()
2020-12-21 13:58:14,200 INFO sqlalchemy.engine.base.Engine COMMIT
2020-12-21 13:58:14,201 INFO sqlalchemy.engine.base.Engine CREATE INDEX ix_temp2_code_pays ON temp2 (code_pays)
2020-12-21 13:58:14,202 INFO sqlalchemy.engine.base.Engine ()
2020-12-21 13:58:14,209 INFO sqlalchemy.engine.base.Engine COMMIT
2020-12-21 13:58:1

# Question 18 : 

Une table appelée sous_nutrition, contenant le nombre de personnes en sous-alimentation pour chaque pays en 2013. Elle devra contenir 4 colonnes : pays, code_pays, année, nb_personnes.

Vous vous en doutez... proposez encore une fois une clé primaire pertinente pour cette table !

In [38]:
#creation d un df
df_ss_al_s = df_ss_al [df_ss_al ['Année'] == '2012-2014' ]   [ [ 'Zone' , 'Code zone' ,  'Année' , 'Pop sous alimente'] ]

#rename
df_ss_al_s.rename ( columns = { 'Zone' : 'pays' , 
                             'Code zone' : 'code_pays' , 
                            'Année' : 'annee' ,  
                             'Pop sous alimente' : 'nb_personne' } , inplace = True)

df_ss_al_s.set_index ( 'code_pays' , inplace = True)


# creation d une table 
df_ss_al_s.to_sql (
    name = 'temp3',
    if_exists = 'replace',
    con = engine ,
    index = True )

#on fixe le code annee et le code_pays 

connection = engine.raw_connection()
c = connection.cursor()

#ALTER TABLE Population RENAME TO popTest;
c.executescript('''
    PRAGMA foreign_keys=off;

    BEGIN TRANSACTION;    
    drop table if exists sous_nutrition;  
    CREATE TABLE sous_nutrition ( 'code_pays' BIGINT,
                            'pays' TEXT,
                            'annee' TEXT,
                            'nb_personne' FLOAT,
                            PRIMARY KEY ("code_pays",  "annee")                          
                            );

   INSERT INTO sous_nutrition SELECT * FROM temp3;    
   DROP TABLE temp3;
   COMMIT TRANSACTION; 
   PRAGMA foreign_keys=on;''')

#close out the connection
c.close()
connection.close()

#Vérification de notre commit
with engine.connect() as conn:
    statement = text("select * from sous_nutrition;")
    result = conn.execute(statement)
    print (result.fetchall())

2020-12-21 13:58:14,937 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("temp3")
2020-12-21 13:58:14,939 INFO sqlalchemy.engine.base.Engine ()
2020-12-21 13:58:14,941 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("temp3")
2020-12-21 13:58:14,942 INFO sqlalchemy.engine.base.Engine ()
2020-12-21 13:58:14,945 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE temp3 (
	code_pays BIGINT, 
	pays TEXT, 
	annee TEXT, 
	nb_personne FLOAT
)


2020-12-21 13:58:14,946 INFO sqlalchemy.engine.base.Engine ()
2020-12-21 13:58:14,953 INFO sqlalchemy.engine.base.Engine COMMIT
2020-12-21 13:58:14,955 INFO sqlalchemy.engine.base.Engine CREATE INDEX ix_temp3_code_pays ON temp3 (code_pays)
2020-12-21 13:58:14,956 INFO sqlalchemy.engine.base.Engine ()
2020-12-21 13:58:14,969 INFO sqlalchemy.engine.base.Engine COMMIT
2020-12-21 13:58:14,973 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-12-21 13:58:14,977 INFO sqlalchemy.engine.base.Engine INSERT INTO temp3 (code_pays, pays, anne

In [39]:
#requete via SQL lite

import sqlite3, sqlalchemy
from sqlalchemy import Table, Column, Integer, String, ForeignKey, MetaData, create_engine, text, inspect
from IPython.display import Markdown, display

engine2 = create_engine('sqlite:///food_sql.db', echo=True)

meta = MetaData()

#q15
#filtrer d un df 
df_pop_s = df_pop[['Code zone' , 'Zone' , 'Année' , 'Valeur' ]]

#rename
df_pop_s.rename (columns = { 'Code zone' : 'code_pays' , 
                            'Zone'  :  'pays' , 
                            'Année' : 'annee' ,
                            'Valeur' : 'population'} , 
                 inplace = True)

df_pop_s['population'] = df_pop_s['population'].apply (lambda x :x * 1000)
df_pop_s.set_index ( 'code_pays' , inplace = True)

# creation d une table
df_pop_s.to_sql (
    name = 'population',
    if_exists = 'replace',
    con = engine2 ,
    index = True , 
)



#q16
#creation d un df
df_s = df[df ['origine'] != 'cereales'][['Code zone' , 'Zone' , 'Année' , 'Produit' , 'Code Produit' , 'origine' ,
           'Disponibilité intérieure (Milliers de tonnes)',
           'Disponibilité alimentaire (Kcal/personne/jour)',
           'Disponibilité alimentaire (kg/personne/an)',
           'Disponibilité de protéines (g/personne/jour)'               ]]

#renamme
df_s.rename (columns = { 'Code zone' : 'code_pays' , 
                        'Zone'  :  'pays' , 
                       'Année' : 'annee' , 
                        'Produit' : 'produit' , 
                        'Code Produit' : 'code_produit' ,
                        'origine' : 'origin' , 
                        'Disponibilité intérieure (Milliers de tonnes)' : 'dispo_alim_tonnes' ,
                        'Disponibilité alimentaire (Kcal/personne/jour)'  : 'dispo_alim_kcal_p_j' ,
                        'Disponibilité alimentaire (kg/personne/an)'    : 'dispo_prot'     ,
                        'Disponibilité de protéines (g/personne/jour)' : 'dispo_mat_gr'    
                       } , inplace = True)

df_s.set_index ( 'code_pays' , inplace = True)

# creation d une table
df_s.to_sql (
    name = 'dispo_alim',
    if_exists = 'replace',
    con = engine2 ,

)



#q17
#creation d un df
df_s = df[df ['origine'] != 'cereales'][['Zone' , 'Code zone' , 'Année' , 
           'Produit' , 'Code Produit' , 
           'Disponibilité intérieure (Milliers de tonnes)' , 
           'Aliments pour animaux (Milliers de tonnes)' , 
           'Semences (Milliers de tonnes)' , 'Pertes (Milliers de tonnes)' , 
           'Traitement (Milliers de tonnes)' ,
           'Nourriture (Milliers de tonnes)' , 
           'Utilisations non alimentaire (Milliers de tonnes)']]

#renamme
df_s.rename (columns = { 
        'Code zone' : 'code_pays' , 
    'Zone'  : 'pays', 
    'Année'  : 'annee',            
    'Produit'  : 'produit', 
    'Code Produit' : 'code_produit' ,           
    'Disponibilité intérieure (Milliers de tonnes)'  : 'dispo_int',            
    'Aliments pour animaux (Milliers de tonnes)'  : 'alim_ani',            
    'Semences (Milliers de tonnes)'  : 'semences', 
    'Pertes (Milliers de tonnes)'  : 'pertes',            
    'Traitement (Milliers de tonnes)'  : 'transfo',           
    'Nourriture (Milliers de tonnes)'  : 'nourriture',            
    'Utilisations non alimentaire (Milliers de tonnes)' : 'autres_utilisations' 
} , inplace = True)


df_s.set_index ( 'code_pays' , inplace = True)


# creation d une table
df_s.to_sql (
    name = 'equilibre_prod',
    if_exists = 'replace',
    con = engine2 ,
    index = True )


#q18
#creation d un df
df_ss_al_s = df_ss_al [df_ss_al ['Année'] == '2012-2014' ]   [ [ 'Zone' , 'Code zone' ,  'Année' , 'Pop sous alimente'] ]

#rename
df_ss_al_s.rename ( columns = { 'Zone' : 'pays' , 
                             'Code zone' : 'code_pays' , 
                            'Année' : 'annee' ,  
                             'Pop sous alimente' : 'nb_personne' } , inplace = True)

df_ss_al_s.set_index ( 'code_pays' , inplace = True)


# creation d une table 
df_ss_al_s.to_sql (
    name = 'sous_nutrition',
    if_exists = 'replace',
    con = engine2 ,
    index = True )

#les clés  : 
    # population : clé composée comprenant le code année et le code pays 
    # dispo_alim : cle composee avec le code annee le code pays et le code produit
    # equilibre_prod : cle composee avec le code annee le code pays et le code produit
    # sous_nutrition : clé composée comprenant le code année et le code pays 



2020-12-21 13:58:15,044 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2020-12-21 13:58:15,047 INFO sqlalchemy.engine.base.Engine ()
2020-12-21 13:58:15,049 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2020-12-21 13:58:15,050 INFO sqlalchemy.engine.base.Engine ()
2020-12-21 13:58:15,051 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("population")
2020-12-21 13:58:15,052 INFO sqlalchemy.engine.base.Engine ()
2020-12-21 13:58:15,055 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("population")
2020-12-21 13:58:15,056 INFO sqlalchemy.engine.base.Engine ()
2020-12-21 13:58:15,060 INFO sqlalchemy.engine.base.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2020-12-21 13:58:15,061 INFO sqlalchemy.engine.base.Engine ()
2020-12-21 13:58:15,064 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_xinfo("population")
2020-12-21 13:58:15,065 INFO sqlalchemy.

<ipython-input-39-7cfb0a743a58>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pop_s['population'] = df_pop_s['population'].apply (lambda x :x * 1000)


2020-12-21 13:58:15,241 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE dispo_alim (
	code_pays BIGINT, 
	pays TEXT, 
	annee BIGINT, 
	produit TEXT, 
	code_produit BIGINT, 
	origin TEXT, 
	dispo_alim_tonnes FLOAT, 
	dispo_alim_kcal_p_j FLOAT, 
	dispo_prot FLOAT, 
	dispo_mat_gr FLOAT
)


2020-12-21 13:58:15,242 INFO sqlalchemy.engine.base.Engine ()
2020-12-21 13:58:15,250 INFO sqlalchemy.engine.base.Engine COMMIT
2020-12-21 13:58:15,252 INFO sqlalchemy.engine.base.Engine CREATE INDEX ix_dispo_alim_code_pays ON dispo_alim (code_pays)
2020-12-21 13:58:15,253 INFO sqlalchemy.engine.base.Engine ()
2020-12-21 13:58:15,260 INFO sqlalchemy.engine.base.Engine COMMIT
2020-12-21 13:58:15,271 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-12-21 13:58:15,506 INFO sqlalchemy.engine.base.Engine INSERT INTO dispo_alim (code_pays, pays, annee, produit, code_produit, origin, dispo_alim_tonnes, dispo_alim_kcal_p_j, dispo_prot, dispo_mat_gr) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
2020-12-21 13:5

2020-12-21 13:58:16,091 INFO sqlalchemy.engine.base.Engine ()
2020-12-21 13:58:16,092 INFO sqlalchemy.engine.base.Engine PRAGMA main.index_list("sous_nutrition")
2020-12-21 13:58:16,093 INFO sqlalchemy.engine.base.Engine ()
2020-12-21 13:58:16,094 INFO sqlalchemy.engine.base.Engine PRAGMA main.index_info("ix_sous_nutrition_code_pays")
2020-12-21 13:58:16,095 INFO sqlalchemy.engine.base.Engine ()
2020-12-21 13:58:16,096 INFO sqlalchemy.engine.base.Engine PRAGMA main.index_list("sous_nutrition")
2020-12-21 13:58:16,097 INFO sqlalchemy.engine.base.Engine ()
2020-12-21 13:58:16,098 INFO sqlalchemy.engine.base.Engine PRAGMA main.index_info("ix_sous_nutrition_code_pays")
2020-12-21 13:58:16,099 INFO sqlalchemy.engine.base.Engine ()
2020-12-21 13:58:16,100 INFO sqlalchemy.engine.base.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = 'sous_nutrition' AND type = 'table'
2020-12-21 13:58:16,101 INFO sqlalchemy.engine.base.Engine ()
20

# Question 19 : 
Écrivez les requêtes SQL permettant de connaître…

 - Les 10 pays ayant le plus haut ratio disponibilité alimentaire/habitant en termes de protéines (en kg) par habitant, 
 - puis en termes de kcal par habitant.
 - Pour l'année 2013, les 10 pays ayant le plus faible ratio disponibilité alimentaire/habitant en termes de protéines (en kg) par habitant.
 - La quantité totale (en kg) de produits perdus par pays en 2013.
 - Les 10 pays pour lesquels la proportion de personnes sous-alimentées est la plus forte.
 - Les 10 produits pour lesquels le ratio Autres utilisations/Disponibilité intérieure est le plus élevé.

In [40]:
#pour l affectation de la clé primaire ce parametre ce fait depuis le logitiel 

#  - Les 10 pays ayant le plus haut ratio disponibilité alimentaire/habitant en termes de protéines (en kg) par habitant, 
with engine.connect() as connection:
    results = connection.execute('''
        SELECT dispo_alim.pays , sum(dispo_prot / 1000) / population as prot 
        FROM dispo_alim , Population
        WHERE dispo_alim.code_pays=Population.code_pays
        GROUP BY dispo_alim.pays
        ORDER BY prot desc 
        limit 10
        ;''')
    print( results.fetchall () )

2020-12-21 13:58:16,176 INFO sqlalchemy.engine.base.Engine 
        SELECT dispo_alim.pays , sum(dispo_prot / 1000) / population as prot 
        FROM dispo_alim , Population
        WHERE dispo_alim.code_pays=Population.code_pays
        GROUP BY dispo_alim.pays
        ORDER BY prot desc 
        limit 10
        ;
2020-12-21 13:58:16,177 INFO sqlalchemy.engine.base.Engine ()
[('Dominique', 1.3988194444444442e-05), ('Bermudes', 1.2933846153846156e-05), ('Saint-Kitts-et-Nevis', 1.0293703703703701e-05), ('Antigua-et-Barbuda', 7.713000000000002e-06), ('Saint-Vincent-et-les Grenadines', 6.7246788990825704e-06), ('Kiribati', 6.541078431372546e-06), ('Grenade', 5.618867924528302e-06), ('Samoa', 4.627473684210528e-06), ('Sao Tomé-et-Principe', 3.4400518134715025e-06), ('Sainte-Lucie', 3.305384615384616e-06)]


In [41]:
#  - puis en termes de kcal par habitant.
with engine.connect() as connection:
    results = connection.execute('''SELECT dispo_alim.pays , sum(dispo_alim_kcal_p_j)/ population as som  
                                 FROM dispo_alim inner join population
                                 GROUP BY dispo_alim.pays 
                                 ORDER BY som desc 
                                 limit 10;''')
    print(results.fetchall())

2020-12-21 13:58:16,208 INFO sqlalchemy.engine.base.Engine SELECT dispo_alim.pays , sum(dispo_alim_kcal_p_j)/ population as som  
                                 FROM dispo_alim inner join population
                                 GROUP BY dispo_alim.pays 
                                 ORDER BY som desc 
                                 limit 10;
2020-12-21 13:58:16,210 INFO sqlalchemy.engine.base.Engine ()
[('Autriche', 0.02159433097669547), ('Belgique', 0.021405308981408747), ('Turquie', 0.021239198743126474), ("États-Unis d'Amérique", 0.02109027232259754), ('Israël', 0.020677860696517412), ('Irlande', 0.02063203718250851), ('Italie', 0.020494566640481802), ('Luxembourg', 0.020276904948939512), ('Égypte', 0.02015089028541503), ('Allemagne', 0.020064971196648337)]


In [42]:
#  - La quantité totale (en kg) de produits perdus par pays en 2013.
with engine.connect() as connection:
    results = connection.execute('''SELECT pays , sum (pertes * 1000) as loss 
                                 FROM equilibre_prod 
                                 GROUP BY pays 
                                 ORDER BY loss desc 
                                 limit 10 ;''')
    print(results.fetchall())

2020-12-21 13:58:19,491 INFO sqlalchemy.engine.base.Engine SELECT pays , sum (pertes * 1000) as loss 
                                 FROM equilibre_prod 
                                 GROUP BY pays 
                                 ORDER BY loss desc 
                                 limit 10 ;
2020-12-21 13:58:19,493 INFO sqlalchemy.engine.base.Engine ()
[('Chine, continentale', 89575000.0), ('Brésil', 75914000.0), ('Inde', 55930000.0), ('Nigéria', 19854000.0), ('Indonésie', 13081000.0), ('Turquie', 12036000.0), ('Mexique', 8289000.0), ('Égypte', 7608000.0), ('Ghana', 7442000.0), ("États-Unis d'Amérique", 7162000.0)]


In [44]:
#  - Les 10 pays pour lesquels la proportion de personnes sous-alimentées est la plus forte.
with engine.connect() as connection:
    results = connection.execute('''SELECT sous_nutrition.pays , nb_personne 
                                 FROM sous_nutrition 
                                 INNER JOIN population
                                 ON sous_nutrition.code_pays = population.code_pays
                                 ORDER BY nb_personne desc 
                                 limit 10;''')
    print(results.fetchall())

2020-12-21 14:11:53,504 INFO sqlalchemy.engine.base.Engine SELECT sous_nutrition.pays , nb_personne 
                                 FROM sous_nutrition 
                                 INNER JOIN population
                                 ON sous_nutrition.code_pays = population.code_pays
                                 ORDER BY nb_personne desc 
                                 limit 10;
2020-12-21 14:11:53,506 INFO sqlalchemy.engine.base.Engine ()
[('Inde', 216300000.0), ('Chine', 137700000.0), ('Chine, continentale', 136600000.0), ('Pakistan', 38100000.0), ('Bangladesh', 26100000.0), ('Éthiopie', 25300000.0), ('Indonésie', 22600000.0), ('République-Unie de Tanzanie', 16399999.999999998), ('Philippines', 14000000.0), ('Nigéria', 13400000.0)]


In [46]:
#  - Les 10 produits pour lesquels le ratio Autres utilisations/Disponibilité intérieure est le plus élevé.
with engine.connect() as connection:
    results = connection.execute('''SELECT pays, produit, dispo_int as dispo, autres_utilisations as util,  autres_utilisations / dispo_int as  ratio 
                                 FROM equilibre_prod 
                                 GROUP BY produit
                                 HAVING  dispo_int  > 0 and autres_utilisations > 0 
                                 ORDER BY ratio desc 
                                 LIMIT 10;''')
    print(results.fetchall())

2020-12-21 14:12:14,911 INFO sqlalchemy.engine.base.Engine SELECT pays, produit, dispo_int as dispo, autres_utilisations as util,  autres_utilisations / dispo_int as  ratio 
                                 FROM equilibre_prod 
                                 GROUP BY produit
                                 HAVING  dispo_int  > 0 and autres_utilisations > 0 
                                 ORDER BY ratio desc 
                                 LIMIT 10;
2020-12-21 14:12:14,912 INFO sqlalchemy.engine.base.Engine ()
[('Afrique du Sud', 'Plantes Aquatiques', 14.0, 14.0, 1.0), ('Afrique du Sud', 'Manioc', 57.0, 57.0, 1.0), ('Afrique du Sud', 'Alcool, non Comestible', 38.0, 38.0, 1.0), ('Afghanistan', 'Huil Plantes Oleif Autr', 361.0, 359.0, 0.9944598337950139), ('Afrique du Sud', 'Huile de Palmistes', 32.0, 16.0, 0.5), ('Afghanistan', 'Huile de Palme', 119.0, 55.0, 0.46218487394957986), ('Afghanistan', 'Huile Graines de Coton', 5.0, 1.0, 0.2)]


In [47]:
#  - Les 10 produits pour lesquels le ratio Autres utilisations/Disponibilité intérieure est le plus élevé.
with engine.connect() as connection:
    results = connection.execute('''SELECT produit, SUM (autres_utilisations) / SUM (dispo_int) as ratio 
                                 FROM equilibre_prod 
                                 WHERE  dispo_int  > 0 and autres_utilisations > 0 
                                 GROUP BY produit
                                 ORDER BY ratio desc 
                                 LIMIT 10;''')
    print(results.fetchall())

2020-12-21 14:12:17,552 INFO sqlalchemy.engine.base.Engine SELECT produit, SUM (autres_utilisations) / SUM (dispo_int) as ratio 
                                 FROM equilibre_prod 
                                 WHERE  dispo_int  > 0 and autres_utilisations > 0 
                                 GROUP BY produit
                                 ORDER BY ratio desc 
                                 LIMIT 10;
2020-12-21 14:12:17,553 INFO sqlalchemy.engine.base.Engine ()
[('Huiles de Foie de Poisso', 1.0), ('Alcool, non Comestible', 1.0), ('Plantes Aquatiques', 0.970664928292047), ('Huiles de Poissons', 0.8961352657004831), ('Girofles', 0.8936170212765957), ('Poivre', 0.8823529411764706), ('Huile de Palmistes', 0.7989290032820867), ('Huil Plantes Oleif Autr', 0.7439429928741093), ('Piments', 0.7391304347826086), ('Huile de Sésame', 0.7109090909090909)]


# Question 20 : 
pour quelques uns des produits identifiés dans cette dernière requête SQL, supposez quelles sont ces "autres utilisations" possibles (recherchez sur internet !)

Les autres utilisations sont :
 - les produits chimiques, cosmetiques, pharmaceutiques, industriel (lubrifiant, peinture, ... ), des hydrocarbures. 



### Livrables 

Voici les livrables attendus, que vous transmettrez dans une archive .ZIP :

Le code en R ou en Python qui vous a permis de calculer chacune des réponses aux questions 1 à 14.
Le code en R, Python et/ou SQL vous ayant permis d'enregistrer les données dans la base de données
Les requêtes SQL vous ayant permis de répondre aux questions 15 à 20. 
Pour faciliter votre passage au jury, déposez sur la plateforme, dans un dossier nommé “P3_nom_prenom”, tous les livrables du projet. Chaque livrable doit être nommé avec le numéro du projet et selon l'ordre dans lequel il apparaît, par exemple “P3_01_coderéponses”, “P3_02_codedonnées”, et ainsi de suite.

### Soutenance

Pour la soutenance, mettez-vous dans la peau du data analyst qui présente les résultats de son étude préliminaire à son équipe (cf paragraphe "Mise en contexte" de cet énoncé).

Voici le déroulé de la soutenance. Même si les durées sont indicatives, merci de traiter toutes les étapes.

 - 7 min : Mise en contexte auprès de votre équipe. Donnez les chiffres clés (nombre de personnes qui décèdent à cause de la faim, chiffres de la sous-nutrition mondiale, évolution possible au cours des années suivantes). Donnez ensuite différentes causes de la faim.


 - 7 min : Approfondissez l'une de ces causes (ou plusieurs de ces causes) en présentant les résultats de vos calculs. 
 Vous conclurez cette partie en répondant à ces 2 questions :

     - la faim dans le monde résulte t'elle d'un manque de production, ou de problèmes technologiques ?
     - quelles sont les prévisions de population en 2050 ? Aura t'on besoin d'augmenter drastiquement la production alimentaire ?
 

Il n'est pas nécessaire de répondre scolairement à chacune des questions 1 à 14 durant la soutenance, mais vous devez présenter les chiffres les plus importants que vous avez trouvé en les incluant dans votre analyse.

 - 3 min : Détaillez des données téléchargées : source, combien de fichiers CSV, à quoi correspondent-ils, à quoi correspond chaque ligne d'un fichier, etc. (il est possible de montrer danc votre présentation des premières lignes des fichiers en les détaillant). Détaillez de la même manière les principales tables (dataframes ou tables sql) utilisées lors de l'analyse.
 - 3 min : Détaillez quelques opération d'algèbre relationnelle utilisées sur les dataframes, dont au moins :

    - une agrégation
    - une jointure (en justifiant le type de jointure : interne, externe gauche, etc.)
    - une restriction
    
Pour chacune des opérations d'algèbre relationnelle détaillée, indiquez les clés primaires des tables avant et après opération. Il conseillé de donner des captures d'écran des tables pour expliquer les opérations utilisées

 - 5 min : Montrez le résultat de chacune des requêtes de la question 19 (sous forme de capture d'écran). Détaillez le code SQL de 2 d'entre elles. Répondez à la question 20. Il n'est pas nécessaire de répondre aux questions 15 à 18 lors de la soutenance.

 - 5 min	Questions-réponses de l'examinateur

### Compétences évaluées

    - Maîtriser les bases de R ou Python
    - Effectuer des requêtes complexes en SQL
    - Appliquer l'algèbre relationnelle en R ou Python
    - Utiliser une documentation technique
    - Récupérer des données à partir d'une source identifiée
    - Utiliser les librairies spécialisées pour la Data Science